# Limpeza dos dados  
Esse notebook estará focado na limpeza e preparação dos dados, para que o notebook de análise fique mais fácil de ler.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import date

In [2]:
pni = pd.read_csv('./dados/dados-brutos/cobertura-vacinal.csv', encoding='ISO-8859-1', 
                  skiprows=3, skipfooter=20 ,sep=';', decimal=',')

gastos_sus = pd.read_csv('./dados/dados-brutos/gastos-sus.csv', encoding='ISO-8859-1', 
                  skiprows=3, skipfooter=12 ,sep=';', decimal=',', thousands='.')

taxa_obito = pd.read_csv('./dados/dados-brutos/taxa-mortalidade-por ano-mes-processamento.csv',
                         encoding='ISO-8859-1', sep=';', skiprows=3, skipfooter=7, decimal=',')

estimativa_populacao = pd.read_excel('./dados/dados-brutos/POP2020_20211117.xls', skiprows=2, 
                                     skipfooter=7, 
                                     names=['Unidade da Federação', 'to_del', 'populacao'] )

/home/johnny/Documentos/dev/data-science/ds/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
pni.head()

,Unidade da Federação,1994,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total
0,11 Rondônia,42.76,50.80,76.05,83.33,93.26,82.31,79.59,83.70,78.18,...,92.75,85.82,76.72,98.18,111.27,63.77,80.74,82.45,51.44,79.88
1,12 Acre,20.53,46.61,36.59,67.56,66.38,85.61,66.79,76.77,81.66,...,84.74,74.35,58.87,59.29,75.54,46.91,62.18,67.44,45.44,69.26
2,13 Amazonas,20.15,35.82,49.88,72.01,70.62,72.19,70.91,70.68,70.98,...,80.09,79.18,67.06,77.14,94.95,48.27,65.33,69.82,49.97,69.65
3,14 Roraima,64.06,88.84,89.18,80.26,97.45,93.76,83.94,72.67,78.33,...,84.42,73.13,60.70,82.30,105.27,57.41,74.26,78.76,48.04,75.68
4,15 Pará,32.44,40.11,51.26,67.09,67.46,82.95,77.23,83.43,82.66,...,84.11,77.87,67.91,71.71,67.51,41.43,57.60,57.44,37.53,73.89


In [4]:
pni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unidade da Federação  27 non-null     object 
 1   1994                  27 non-null     object 
 2   1995                  27 non-null     object 
 3   1996                  27 non-null     object 
 4   1997                  27 non-null     float64
 5   1998                  27 non-null     float64
 6   1999                  27 non-null     float64
 7   2000                  27 non-null     float64
 8   2001                  27 non-null     float64
 9   2002                  27 non-null     float64
 10  2003                  27 non-null     float64
 11  2004                  27 non-null     float64
 12  2005                  27 non-null     float64
 13  2006                  27 non-null     float64
 14  2007                  27 non-null     float64
 15  2008                  27 

### Abreviação da Unidade de Federação  
Para facilitar de leitura posteriormente, vamos abreviar os nomes dos estados também.

Essas modificações já são suficientes para trabalharmos. Agora vamos olhar as outras bases de dados.

In [5]:
siglas = {'Acre': 'AC', 'Alagoas' : 'AL', 'Amazonas' : 'AM', 'Amapá' : 'AP',
         'Bahia' : 'BA', 'Ceará': 'CE', 'Distrito Federal' : 'DF', 'Espírito Santo': 'ES',
         'Goiás': 'GO', 'Maranhão': 'MA', 'Mato Grosso' : 'MT', 'Mato Grosso do Sul' : 'MS',
         'Minas Gerais' : 'MG', 'Pará' : 'PA', 'Paraná': 'PR', 'Paraíba' : 'PA', 
         'Pernambuco' : 'PE', 'Piauí' : 'PI', 'Rio de Janeiro' : 'RJ', 
         'Rio Grande do Norte' : 'RN', 'Rio Grande do Sul' : 'RS', 'Rondônia' : 'RO', 
         'Roraima' : 'RR', 'Santa Catarina' : 'SC', 'São Paulo' : 'SP', 
         'Sergipe' : 'SE', 'Tocantins' : 'TO'}


In [6]:
def abreviar_uf(df : pd.DataFrame()):
    df['Unidade da Federação'] = df['Unidade da Federação'].str.split(' ', 1).str[1]
    df['uf'] = df['Unidade da Federação'].map(siglas)
    df.drop('Unidade da Federação', axis=1, inplace=True)
    
    lista_colunas = list(df.columns)
    lista_colunas.pop()
    lista_colunas.insert(0, 'uf')
        
    df = df.reindex(columns=lista_colunas)
    
    return df
    

In [7]:
pni = abreviar_uf(pni)
gastos_sus = abreviar_uf(gastos_sus)
taxa_obito = abreviar_uf(taxa_obito)

Para o dataset de Estimativa não podemos aplicar a mesma função, pois a coluna da UF já está sem os números.   
Portanto o tratamento desse base foi feita manualmente.

In [8]:
estimativa_populacao['uf'] = estimativa_populacao['Unidade da Federação'].map(siglas)
estimativa_populacao.drop([0, 8, 18, 23, 27], axis=0, inplace=True)
estimativa_populacao.drop(['Unidade da Federação', 'to_del'], axis=1, inplace=True)

estimativa_populacao = estimativa_populacao.reindex(columns=['uf', 'populacao'])
estimativa_populacao.reset_index()

estimativa_populacao

,uf,populacao
1,RO,1796460
2,AC,894470
3,AM,4207714
4,RR,631181
5,PA,8690745
6,AP,861773
7,TO,1590248
9,MA,7114598
10,PI,3.281.480(1)
11,CE,9.187.103(1)


Outra correção necessária é converter para número as linhas que possuem parênteses e está fazendo a coluna toda ser considerada uma string.

In [9]:
correcao = {'3.281.480(1)' : 3281480, '9.187.103(1)' : 9187103, '9.616.621(2)' : 9616621, 
            '3.351.543(2)' : 3351543, '2.318.822(3)' : 2318822, '14.930.634(3)' : 14930634, 
            '7.113.540(4)' : 7113540, '3.055.149(4)' : 3055149}

estimativa_populacao['populacao'] = estimativa_populacao['populacao'].apply(lambda x: 
                                                        correcao[x] if x in correcao else x)

In [10]:
estimativa_populacao

,uf,populacao
1,RO,1796460
2,AC,894470
3,AM,4207714
4,RR,631181
5,PA,8690745
6,AP,861773
7,TO,1590248
9,MA,7114598
10,PI,3281480
11,CE,9187103


### Filtrando os períodos

Vemos que as 4 primeiras colunas são do tipo string. O primeiro, se trata da UF então é normal, mas os três em seguida deveriam ser float. Isso se dá porque algumas linhas estão com valores nulos. Portanto essas linhas serão cortados.  
E no final das colunas temos o Total e 2019 - que segundo própria DATASUS os dados estão atualizados somente até setembro/2019, portanto será retirado também.

In [11]:
pni = pni.set_index('uf')

In [12]:
pni = pni.iloc[:, 16:-2]


In [13]:
def delimitar_dataset(df : pd.DataFrame()):
    df = df.set_index('uf')
    df = df.iloc[:, 24:-34] 
    
    
    return df

In [14]:
taxa_obito = delimitar_dataset(taxa_obito)
gastos_sus = delimitar_dataset(gastos_sus)

### Convertendo para Milhões (MM)  
Os gastos do Sus vamos dividir por milhão para facilitar a leitura.

In [15]:
colunas_numericas = list(gastos_sus.select_dtypes(include='number'))

gastos_sus[colunas_numericas] = gastos_sus[colunas_numericas].apply(lambda x: x / 1e6). \
                                    astype('float')

gastos_sus.columns

Index(['2010/Jan', '2010/Fev', '2010/Mar', '2010/Abr', '2010/Mai', '2010/Jun',
       '2010/Jul', '2010/Ago', '2010/Set', '2010/Out',
       ...
       '2018/Mar', '2018/Abr', '2018/Mai', '2018/Jun', '2018/Jul', '2018/Ago',
       '2018/Set', '2018/Out', '2018/Nov', '2018/Dez'],
      dtype='object', length=108)

In [16]:
#Existia um dado na linha do estado de AC que estava preenchida com um '-', e foi necessário 
#tratar manualmente para que o dataset todo não fosse considerado como string
#No lugar dele foi colocado o mesmo valor do mês anterior.

gastos_sus.iloc[1, 20] = 2.307579

In [17]:
gastos_sus = gastos_sus.astype('float')

### Consolidar os numeros mensais para anuais

Outro ponto é que como os dados do pni são anuais, precisamos que os demais datasets estejam dessa forma. Para tanto vamos fazer a conversão delas também. 
E precisamos retirar a coluna de total também.

In [18]:
gastos_sus_mensal = gastos_sus.T
taxa_obito_mensal = taxa_obito.T

#Como vamos aplicar o melt para os dados do pni também, vamos inverter a matriz dele agora.
pni = pni.T

In [19]:
gastos_sus_mensal.head()

uf,RO,AC,AM,RR,PA,AP,TO,MA,PI,CE,...,ES,RJ,SP,PR,SC,RS,MS,MT,GO,DF
2010/Jan,3.684407,2.491042,9.419461,1.437341,25.841890,1.849157,6.411313,18.906152,13.377546,31.949416,...,15.195271,47.245572,214.565775,65.492819,35.965111,66.304169,12.148433,12.364761,23.352380,14.152357
2010/Fev,3.623016,1.954474,8.680179,1.332094,26.527192,1.605246,6.403263,17.540838,12.026088,33.250426,...,14.089292,50.655915,203.116173,62.237724,30.499829,61.527420,11.270973,11.382133,23.520045,12.797956
2010/Mar,3.890462,2.389996,9.168055,1.900514,27.433129,1.525752,6.698841,19.741933,13.110929,35.653268,...,14.860460,56.116555,227.248380,70.702538,35.106629,62.490522,11.947693,11.947040,25.369726,14.766109
2010/Abr,4.193534,2.654879,9.897902,1.933875,25.753868,1.572809,6.905354,19.445209,13.027828,35.280278,...,14.590678,52.070900,231.543599,70.374694,38.016216,66.814413,11.435566,12.233218,24.404417,14.251503
2010/Mai,4.534401,2.844085,9.651362,1.773958,27.515464,1.653305,6.864576,20.174079,13.209893,35.760072,...,15.551855,59.396969,230.188803,72.058181,36.075963,67.900518,12.392542,12.656421,24.987746,13.992322


In [20]:
taxa_obito_mensal

uf,RO,AC,AM,RR,PA,AP,TO,MA,PI,CE,...,ES,RJ,SP,PR,SC,RS,MS,MT,GO,DF
2010/Jan,1.70,2.45,2.07,2.52,1.87,2.99,2.62,1.51,1.70,2.94,...,3.67,5.32,4.97,3.56,4.22,4.54,3.54,3.35,2.61,2.70
2010/Fev,1.29,1.94,1.65,2.58,1.66,2.73,2.65,1.46,1.80,3.18,...,3.35,5.58,4.78,3.55,4.05,4.90,3.41,3.29,2.53,3.17
2010/Mar,1.48,2.74,1.77,2.05,1.61,2.28,2.78,1.54,1.75,2.90,...,3.63,5.14,4.48,3.48,4.07,4.37,3.25,2.93,2.45,2.43
2010/Abr,1.64,3.05,1.79,1.88,1.68,2.96,2.19,1.46,1.94,3.04,...,3.33,5.28,4.56,3.37,3.54,4.19,3.40,3.09,2.27,2.43
2010/Mai,2.05,2.46,1.86,2.25,1.48,1.49,2.25,1.44,1.67,2.94,...,3.26,5.12,4.40,3.48,3.59,4.36,3.38,3.10,2.16,2.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018/Ago,2.59,3.05,3.08,2.80,2.43,2.62,3.50,2.25,2.49,3.82,...,3.64,6.82,5.08,3.98,4.70,5.95,4.14,3.68,3.28,3.44
2018/Set,2.65,3.82,2.90,3.66,2.32,2.26,3.13,2.11,2.70,3.88,...,3.74,6.88,5.08,3.87,4.47,5.74,3.96,3.83,3.26,2.97
2018/Out,2.58,3.49,3.19,2.91,2.22,2.31,3.28,2.03,2.72,3.71,...,3.82,6.56,4.81,3.75,4.08,5.28,4.17,3.29,3.60,2.90
2018/Nov,2.82,5.00,3.08,3.07,2.16,2.77,3.06,2.18,2.88,3.79,...,3.52,6.49,4.91,3.80,4.03,5.15,3.91,3.29,3.33,3.50


In [21]:
meses = {
    'Jan': 1, 'Fev': 2, 'Mar': 3,
    'Abr': 4, 'Mai': 5, 'Jun': 6,
    'Jul': 7, 'Ago': 8, 'Set': 9,
    'Out': 10, 'Nov': 11, 'Dez': 12    
}


def para_dia(ano_mes : str):
    ano : int = int(ano_mes[:4])
    mes : str = ano_mes[5:]
    mes_inteiro : int = meses[mes]
    
    return date(ano, mes_inteiro ,1)

gastos_sus_mensal.index = gastos_sus_mensal.index.map(para_dia)
taxa_obito_mensal.index = taxa_obito_mensal.index.map(para_dia)

In [22]:
pni

uf,RO,AC,AM,RR,PA,AP,TO,MA,PI,CE,...,ES,RJ,SP,PR,SC,RS,MS,MT,GO,DF
2010,79.26,76.32,73.30,75.19,81.55,75.31,75.12,87.04,75.44,74.40,...,74.74,67.48,70.36,75.22,74.18,67.22,75.77,73.54,76.25,80.07
2011,92.75,84.74,80.09,84.42,84.11,77.88,87.78,87.15,86.04,83.44,...,88.35,83.91,84.96,89.97,88.07,81.99,89.37,87.40,96.24,67.70
2012,85.82,74.35,79.18,73.13,77.87,73.77,76.34,77.67,77.57,74.76,...,79.71,73.71,77.28,79.48,78.79,72.95,83.46,81.37,82.52,64.84
2013,76.72,58.87,67.06,60.70,67.91,67.66,72.78,73.54,71.99,73.09,...,72.30,68.11,73.37,77.59,72.94,73.47,81.81,75.04,79.36,85.83
2014,98.18,59.29,77.14,82.30,71.71,76.11,85.09,83.27,76.06,96.57,...,90.98,84.08,88.20,86.95,91.81,84.15,110.11,92.61,86.63,89.33
2015,111.27,75.54,94.95,105.27,67.51,88.75,92.37,94.40,80.68,107.71,...,98.39,96.14,98.63,96.41,100.69,87.69,113.07,100.32,93.55,71.55
2016,63.77,46.91,48.27,57.41,41.43,56.62,60.94,43.38,46.94,56.44,...,51.19,47.98,45.94,55.32,58.88,53.86,63.55,58.32,53.60,75.28
2017,80.74,62.18,65.33,74.26,57.60,58.66,81.70,60.87,67.25,73.67,...,66.35,67.04,67.29,77.66,75.82,72.54,78.81,74.28,73.25,73.82
2018,82.45,67.44,69.82,78.76,57.44,63.32,82.32,64.51,70.80,77.37,...,74.22,61.37,71.52,81.79,82.43,74.94,83.52,76.70,71.85,73.85


In [23]:
def derreter_colunas(df):
    df = df.reset_index().melt(id_vars=['index'], value_vars=df.columns)
    return df

In [24]:
gastos_sus_mensal = derreter_colunas(gastos_sus_mensal)
taxa_obito_mensal = derreter_colunas(taxa_obito_mensal)
pni = derreter_colunas(pni)

In [25]:
def criar_coluna_ano(df):
    df['index'] = pd.to_datetime(df['index'])
    df['ano'] = df['index'].dt.year
    
    

In [26]:
criar_coluna_ano(taxa_obito_mensal)
criar_coluna_ano(gastos_sus_mensal)

In [27]:
taxa_obito_anual = taxa_obito_mensal.groupby(['uf', 'ano']).sum().reset_index()
gastos_sus_anual = gastos_sus_mensal.groupby(['uf', 'ano']).sum().reset_index()

In [28]:
taxa_obito_anual.head()

,uf,ano,value
0,AC,2010,32.70
1,AC,2011,34.93
2,AC,2012,28.82
3,AC,2013,29.36
4,AC,2014,28.00


In [29]:
gastos_sus_anual.head()

,uf,ano,value
0,AC,2010,28.773343
1,AC,2011,32.973851
2,AC,2012,32.929373
3,AC,2013,33.605894
4,AC,2014,36.039320


In [30]:
gastos_sus_anual.query('ano==[2010]')

,uf,ano,value
0,AC,2010,28.773343
9,AL,2010,142.837204
18,AM,2010,114.040833
27,AP,2010,20.736477
36,BA,2010,616.650969
45,CE,2010,411.994715
54,DF,2010,165.987601
63,ES,2010,185.140020
72,GO,2010,302.067940
81,MA,2010,237.304511


In [31]:
pni.head()

,index,uf,value
0,2010,RO,79.26
1,2011,RO,92.75
2,2012,RO,85.82
3,2013,RO,76.72
4,2014,RO,98.18


Agora podemos salvar essas bases de dados.

In [32]:
taxa_obito_anual.to_csv('./dados/dados-limpos/taxa-obito-limpo.csv', index=False)
gastos_sus_anual.to_csv('./dados/dados-limpos/gastos-sus-limpo.csv', index=False)
pni.to_csv('./dados/dados-limpos/cobertura-vacinal-limpo.csv', index=False)